In [2]:
#import der benötigten Python Libraries 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import time
import datetime

import re
import emoji

import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', 500)

#um die häufigsten Wörter im Wahlkampf anzeigen lassen zu können > wird zur tokenisierung etc benötigt
import nltk 
from nltk.corpus import stopwords
from nltk import FreqDist

import spacy

In [4]:
#DataFrames der AFD

df_afd_raw = pd.read_excel("../data/Parteien/@AfD_user_tweets_bearbeitet.xlsx")
df_afd_raw.rename(columns={'Text': "original_tweets"}, inplace=True)
tweets_df_clean = df_afd_raw[['Name', 'original_tweets']]

In [3]:
import string 

def clean_text(text):
#konvertiert text zu lowercase  
 text = text.lower()
#löscht text in eckigen Klammern
 text = re.sub(r'\[.*?\]', '', text)
#löscht Punktuation + Links + @Zeichen 
 text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) 
#löscht Wörter die Nummern enthalten
 text = re.sub(r'\w*\d\w*', '', text)
#löscht Anführungsstriche
 text = re.sub(r'„', '', text)
 text = re.sub(r'“', '', text)
 return text

def give_emoji_free_text(text): #Emojis werden aus den tweets entfernt 
    return emoji.get_emoji_regexp().sub(r'',text)

tweets_df_clean['emoji_free_tweets'] = tweets_df_clean['original_tweets'].apply(give_emoji_free_text)

#Create a new column with emji free text 
tweets_df_clean['clean_tweets'] = tweets_df_clean['emoji_free_tweets'].apply(clean_text)                               
                               
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets
0,Alternative für 🇩🇪 Deutschland,Phänomenal! In #Sachsen hängt unsere #AfD alle anderen Parteien mit MEHR ALS 10 % Abstand bei den Zweitstimmen ab! #abernormal #AfD \nhttps://t.co/wVGDWQJGDU https://t.co/2Gii3GUcnE,Phänomenal! In #Sachsen hängt unsere #AfD alle anderen Parteien mit MEHR ALS 10 % Abstand bei den Zweitstimmen ab! #abernormal #AfD \nhttps://t.co/wVGDWQJGDU https://t.co/2Gii3GUcnE,phänomenal in sachsen hängt unsere afd alle anderen parteien mit mehr als abstand bei den zweitstimmen ab abernormal afd \nhttpstcowvgdwqjgdu
1,Alternative für 🇩🇪 Deutschland,"""Wir können auf eine solide Stammwählerschaft zählen!"" #afd #abernormal #weidel https://t.co/EQ8OkdqQ6K","""Wir können auf eine solide Stammwählerschaft zählen!"" #afd #abernormal #weidel https://t.co/EQ8OkdqQ6K",wir können auf eine solide stammwählerschaft zählen afd abernormal weidel
2,Alternative für 🇩🇪 Deutschland,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei https://t.co/GGq847K9BS,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei https://t.co/GGq847K9BS,genial die prognose zur sieht die afd bei und somit als zweitstärkste kraft im osten abernormal volkspartei
3,Alternative für 🇩🇪 Deutschland,"#ARD-Hochrechnung 19:59 Uhr \n\n#AfD: 10,9 % #abernormal\n#CDU: 24,5 %\n#SPD: 25,5 %\n#FDP: 11,7 %\n#Grüne: 13,8 %\n#Linke: 5 %","#ARD-Hochrechnung 19:59 Uhr \n\n#AfD: 10,9 % #abernormal\n#CDU: 24,5 %\n#SPD: 25,5 %\n#FDP: 11,7 %\n#Grüne: 13,8 %\n#Linke: 5 %",ardhochrechnung uhr \n\nafd abernormal\ncdu \nspd \nfdp \ngrüne \nlinke
4,Alternative für 🇩🇪 Deutschland,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei\n\nQuelle: Screenshot ARD https://t.co/v5NQWcdMWf,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei\n\nQuelle: Screenshot ARD https://t.co/v5NQWcdMWf,genial die prognose zur sieht die afd bei und somit als zweitstärkste kraft im osten abernormal volkspartei\n\nquelle screenshot ard
...,...,...,...,...
355,Alternative für 🇩🇪 Deutschland,"Die stellvertretende #AfD-Bundessprecherin @Beatrix_vStorch macht deutlich, wie verrückt die grüne Gedankenwelt rund um sexuelle Identitäten ist. #AfD #abernormal https://t.co/iay2ZmO4I7","Die stellvertretende #AfD-Bundessprecherin @Beatrix_vStorch macht deutlich, wie verrückt die grüne Gedankenwelt rund um sexuelle Identitäten ist. #AfD #abernormal https://t.co/iay2ZmO4I7",die stellvertretende afdbundessprecherin beatrixvstorch macht deutlich wie verrückt die grüne gedankenwelt rund um sexuelle identitäten ist afd abernormal
356,Alternative für 🇩🇪 Deutschland,Wir haben die Halbe-Million-Marke geknackt! Unsere #AfD-Spendenrakete hebt dank ihrer Unterstützung weiter ab! Wir sagen DANKE! 💙 #BTW2021 #abernormal\nStand 28.06.2021: 501.218 €!\nJetzt unterstützen:\nhttps://t.co/dLyhLgCliv https://t.co/7gTDvrD20K,Wir haben die Halbe-Million-Marke geknackt! Unsere #AfD-Spendenrakete hebt dank ihrer Unterstützung weiter ab! Wir sagen DANKE! #BTW2021 #abernormal\nStand 28.06.2021: 501.218 €!\nJetzt unterstützen:\nhttps://t.co/dLyhLgCliv https://t.co/7gTDvrD20K,wir haben die halbemillionmarke geknackt unsere afdspendenrakete hebt dank ihrer unterstützung weiter ab wir sagen danke abernormal\nstand €\njetzt unterstützen\nhttpstcodlyhlgcliv
357,Alternative für 🇩🇪 Deutschland,KopfhörerFeuer Ausgabe 25/21 unseres #Podcast s - jetzt reinhören! 7 Tage #Deutschland – der #AfD-Wochenendpodcast mit vielen spannenden Themen!\n\n▶ https://t.co/xX7koQ1w5Z\n▶ https://t.co/EwnuuDduB4\n▶ https://t.co/cC1WqE8qTt https://t.co/E4nRd2dUti,KopfhörerFeuer Ausgabe 25/21 unseres #Podcast s - jetzt reinhören! 7 Tage #Deutschland – der #AfD-Wochenendpodcast mit vielen spannenden Themen!\n\n https://t.co/xX7koQ1w5Z\n

In [4]:
data = list(tweets_df_clean.loc[:, "clean_tweets"].values) ## data ist ein doc

In [5]:
# Die Wörter werden lemmatisiert -> ihr Ursprungsform wird betracht (ungebeugt)
# durch den Parameter "allowed_postags" filtern wir den Text und lassen nur Substatntive und Adjektive ausgeben
def lemmatization(texts, allowed_postags=["NOUN","ADJ"]):
    nlp = spacy.load("de_core_news_lg", disable=["parser", "ner"]) 
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:120])

afd ander parteien abstand zweitstimmen


In [6]:
tweets_df_clean['clean_tweets_noun'] = lemmatized_texts

In [7]:
tweets_df_clean

,Name,original_tweets,emoji_free_tweets,clean_tweets,clean_tweets_noun
0,Alternative für 🇩🇪 Deutschland,Phänomenal! In #Sachsen hängt unsere #AfD alle anderen Parteien mit MEHR ALS 10 % Abstand bei den Zweitstimmen ab! #abernormal #AfD \nhttps://t.co/wVGDWQJGDU https://t.co/2Gii3GUcnE,Phänomenal! In #Sachsen hängt unsere #AfD alle anderen Parteien mit MEHR ALS 10 % Abstand bei den Zweitstimmen ab! #abernormal #AfD \nhttps://t.co/wVGDWQJGDU https://t.co/2Gii3GUcnE,phänomenal in sachsen hängt unsere afd alle anderen parteien mit mehr als abstand bei den zweitstimmen ab abernormal afd \nhttpstcowvgdwqjgdu,afd ander parteien abstand zweitstimmen
1,Alternative für 🇩🇪 Deutschland,"""Wir können auf eine solide Stammwählerschaft zählen!"" #afd #abernormal #weidel https://t.co/EQ8OkdqQ6K","""Wir können auf eine solide Stammwählerschaft zählen!"" #afd #abernormal #weidel https://t.co/EQ8OkdqQ6K",wir können auf eine solide stammwählerschaft zählen afd abernormal weidel,solide stammwählerschaft afd weidel
2,Alternative für 🇩🇪 Deutschland,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei https://t.co/GGq847K9BS,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei https://t.co/GGq847K9BS,genial die prognose zur sieht die afd bei und somit als zweitstärkste kraft im osten abernormal volkspartei,prognose afd zweitstärkste kraft osten volkspartei
3,Alternative für 🇩🇪 Deutschland,"#ARD-Hochrechnung 19:59 Uhr \n\n#AfD: 10,9 % #abernormal\n#CDU: 24,5 %\n#SPD: 25,5 %\n#FDP: 11,7 %\n#Grüne: 13,8 %\n#Linke: 5 %","#ARD-Hochrechnung 19:59 Uhr \n\n#AfD: 10,9 % #abernormal\n#CDU: 24,5 %\n#SPD: 25,5 %\n#FDP: 11,7 %\n#Grüne: 13,8 %\n#Linke: 5 %",ardhochrechnung uhr \n\nafd abernormal\ncdu \nspd \nfdp \ngrüne \nlinke,ardhochrechnung uhr afd grüne link
4,Alternative für 🇩🇪 Deutschland,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei\n\nQuelle: Screenshot ARD https://t.co/v5NQWcdMWf,Genial! Die Prognose zur #BTW21 sieht die #AfD bei 21 % und somit als zweitstärkste Kraft im Osten! #abernormal #Volkspartei\n\nQuelle: Screenshot ARD https://t.co/v5NQWcdMWf,genial die prognose zur sieht die afd bei und somit als zweitstärkste kraft im osten abernormal volkspartei\n\nquelle screenshot ard,prognose afd zweitstärkste kraft osten volkspartei quelle screenshot
...,...,...,...,...,...
355,Alternative für 🇩🇪 Deutschland,"Die stellvertretende #AfD-Bundessprecherin @Beatrix_vStorch macht deutlich, wie verrückt die grüne Gedankenwelt rund um sexuelle Identitäten ist. #AfD #abernormal https://t.co/iay2ZmO4I7","Die stellvertretende #AfD-Bundessprecherin @Beatrix_vStorch macht deutlich, wie verrückt die grüne Gedankenwelt rund um sexuelle Identitäten ist. #AfD #abernormal https://t.co/iay2ZmO4I7",die stellvertretende afdbundessprecherin beatrixvstorch macht deutlich wie verrückt die grüne gedankenwelt rund um sexuelle identitäten ist afd abernormal,stellvertretende afdbundessprecherin grüne gedankenwelt sexuelle identitäten afd
356,Alternative für 🇩🇪 Deutschland,Wir haben die Halbe-Million-Marke geknackt! Unsere #AfD-Spendenrakete hebt dank ihrer Unterstützung weiter ab! Wir sagen DANKE! 💙 #BTW2021 #abernormal\nStand 28.06.2021: 501.218 €!\nJetzt unterstützen:\nhttps://t.co/dLyhLgCliv https://t.co/7gTDvrD20K,Wir haben die Halbe-Million-Marke geknackt! Unsere #AfD-Spendenrakete hebt dank ihrer Unterstützung weiter ab! Wir sagen DANKE! #BTW2021 #abernormal\nStand 28.06.2021: 501.218 €!\nJetzt unterstützen:\nhttps://t.co/dLyhLgCliv https://t.co/7gTDvrD20K,wir haben die halbemillionmarke geknackt unsere afdspendenrakete hebt dank ihrer unterstützung weiter ab wir sagen danke abernormal\nstand €\njetzt unterstützen\nhttpstcodlyhlgcliv,halbemillionmarke afdspendenrakete unterstützung €
357,Alternative für 🇩🇪 Deutschland,KopfhörerFeuer Ausgabe 25/21 unseres #Pod

In [8]:
#stopwords werden entfernt 
#sätze werden in einzelne Wörter heruntergebochen 

import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('german')
stop_words.extend(['amp','mehr erfahren','erfahren','afd','tinochrupalla'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets_df_clean.clean_tweets_noun.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(tweets_df_clean.clean_tweets_noun, 10)
unigram = pd.DataFrame(common_words, columns = ['unigram' , 'count'])

In [10]:
unigram

,unigram,count
0,september,24
1,deutsch,22
2,bürger,20
3,grüne,17
4,euro,16
5,tage,15
6,politik,15
7,uhr,14
8,wahlkampfveranstaltung,13
9,ausgabe,12


In [11]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2,2),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(tweets_df_clean.clean_tweets_noun, 10)
bigram = pd.DataFrame(common_words, columns = ['bigram' , 'count'])

In [12]:
bigram

,bigram,count
0,ausgabe podcast,9
1,tage afdwochenendpodcast,9
2,afdwochenendpodcast spannend,9
3,spannend themen,9
4,podcast tage,8
5,herzlich einladung,7
6,uhr wahlkampfveranstaltung,7
7,millionen euro,6
8,einladung september,6
9,september uhr,6


In [13]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3,3),stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(tweets_df_clean.clean_tweets_noun, 10)
trigram = pd.DataFrame(common_words, columns = ['trigram' , 'count'])

In [14]:
trigram

,trigram,count
0,tage afdwochenendpodcast spannend,9
1,afdwochenendpodcast spannend themen,9
2,ausgabe podcast tage,8
3,podcast tage afdwochenendpodcast,8
4,herzlich einladung september,6
5,einladung september uhr,6
6,september uhr wahlkampfveranstaltung,6
7,wahlkampfveranstaltung herzlich einladung,5
8,stimme normal grundlegend,3
9,normal grundlegend weisen,3


In [15]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(
                        analyzer='word', 
                        ngram_range=(1,2),#bigramme werden berücksichtigt 
                        max_df=0.8,
                        min_df=2,# minimales Vorkommen eines Wortes
                        stop_words= stop_words,#stopwords werden entfernt 
                        lowercase=True #lowercase 
                        )
data_matrix = vectorizer.fit_transform(tweets_df_clean.clean_tweets_noun)
data_matrix

<360x448 sparse matrix of type '<class 'numpy.int64'>'
	with 1511 stored elements in Compressed Sparse Row format>

In [25]:
lda_model = LatentDirichletAllocation(
n_components=5, # Nummer von Topics
learning_method='batch',
random_state=20,       
n_jobs = -1  # Alle verfügbaren CPUs werden verwendet 
                                     )
lda_output = lda_model.fit_transform(data_matrix)

In [26]:
for i,topic in enumerate(lda_model.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')


Top 10 words for topic #0:
['prozent', 'grün', 'migration', 'wochen', 'hoch', 'afdbundestagsabgeordnete', 'bundestag', 'jahre', 'september', 'deutsch']


Top 10 words for topic #1:
['jahren', 'millionen euro', 'flutkatastrophe', 'millionen', 'aktuell', 'bundesregierung', 'politik', 'grüne', 'euro', 'bürger']


Top 10 words for topic #2:
['jahren', 'neue', 'beitrag', 'stunden', 'groß', 'afghanen', 'bundestagswahl', 'thema', 'zeit', 'stimme']


Top 10 words for topic #3:
['podcast', 'afdwochenendpodcast', 'spannend', 'spannend themen', 'tage afdwochenendpodcast', 'afdwochenendpodcast spannend', 'themen', 'spitzenkandidat', 'ausgabe', 'tage']


Top 10 words for topic #4:
['september uhr', 'einladung september', 'gast', 'uhr wahlkampfveranstaltung', 'herzlich', 'herzlich einladung', 'einladung', 'september', 'uhr', 'wahlkampfveranstaltung']




In [27]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_matrix, vectorizer, mds='tsne')

/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/philipp/opt/anaconda3/lib/python3.9/site-packages/past/builti

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1      150.222153  -72.269554       1        1  23.295365
4       51.163364 -159.968018       2        1  21.735025
0      -73.144554   21.523815       3        1  18.665096
2      -66.410133 -110.606438       4        1  18.202460
3       54.417324   12.443444       5        1  18.102053, topic_info=                Term       Freq      Total Category  logprob  loglift
86           deutsch  17.000000  17.000000  Default  30.0000  30.0000
79            bürger  16.000000  16.000000  Default  29.0000  29.0000
370             tage  12.000000  12.000000  Default  28.0000  28.0000
114             euro  13.000000  13.000000  Default  27.0000  27.0000
51           ausgabe  10.000000  10.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
89   diskriminierung   2.509781   4.704913   Topic5  -4.7428   1.0807
12            afdbus   2.375447   4.747644   Topic5  -4.7978   1.0167
39       altparteien   2.501806   9.292730   Topic5  -4.7460   0.3969
286          politik   1.852129  12.488970   Topic5  -5.0466  -0.1994
243         lockdown   1.724059   3.886816   Topic5  -5.1183   0.8962

[252 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
2         3  0.256640           abernormal
2         5  0.513280           abernormal
9         2  0.421139    afdbundessprecher
9         4  0.210569    afdbundessprecher
9         5  0.210569    afdbundessprecher
...     ...       ...                  ...
436       1  0.467892               zahlen
436       5  0.467892               zahlen
439       4  0.931059                 zeit
446       4  0.865766        zweitstärkste
447       4  0.865766  zweitstärkste kraft

[293 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 3, 4])